# Project Name
This project notebook intends to predict, Given a dataset of clients, which is more likely to accept and invite and buy
the companies product.
We will demonstrate and explain the analysis and explain our method of work.

Install relevant packages.

In [1]:
%pip install scikit-learn
%pip install xgboost
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ofeks\OneDrive\תואר_שני\שנה ב\סמסטר א\נושאים מתקדמים במדעי הנתונים למנהל עסקים\temp_code\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ofeks\OneDrive\תואר_שני\שנה ב\סמסטר א\נושאים מתקדמים במדעי הנתונים למנהל עסקים\temp_code\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ofeks\OneDrive\תואר_שני\שנה ב\סמסטר א\נושאים מתקדמים במדעי הנתונים למנהל עסקים\temp_code\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Imports

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.utils import class_weight
import pandas as pd
import xgboost as xgb
from tqdm import tqdm


### First Step
In order to use the 'reviews' data sets in our complete prediction model. We will need to predict the raiting of each review
the same way we did in the bonus assignment. The predicted rating will help us in taking advantage of the review data.
- The model will be built the same way as it was built in the bonus assignment.
- The data used will be the data given in the bonus assignment.
- The trained model will be used to predict the rating in the reviews_rollout.csv and reviews_training.csv
- The act of training the model on data X and predicting on data was is valid becuase the populations are the same.

In [3]:

def train_bonus_ass_model():
    """
        The function repeats the training process of the model in the
        Bonus assignment. Returned values are the trained model and the selected values.

        Features selection is done by '''SelectPercentile'''.
        Model used is XGBoost.
        Test is 20% of total data.
    """
    df = pd.read_csv("../BonusAssignment/text_training.csv", usecols=list(range(1,2002)))

    # separate the features and target variable
    features = df.iloc[:, 1:-1] # all columns except the last one (rating)
    labels = df.iloc[:, -1] # last column (rating)

    selector = SelectPercentile(percentile=10)
    x = selector.fit_transform(features, labels)
    selected_features = selector.get_feature_names_out()

    x_train, x_test, y_train, y_test = train_test_split(x, labels, test_size=0.2, random_state=1)

    xgb_bonus_clf = xgb.XGBClassifier(max_depth=4, seed=2)
    xgb_bonus_clf.fit(x_train, y_train)

    return xgb_bonus_clf, selected_features

In [4]:
def bonus_ass_rating_prediction(table_path, trained_model, select_features):
    """
        The function Receives the path to one of the foloowing tables: 'reviews_rollout.csv'
        or 'reviews_training.csv' and returns the predicted values in dataframe along with the
        respected ids.

        Assumptions:
        1. Both tables contain an id column
        2. Rating Column does not appear in any table
        3. model was Trained on the same tryp of population
    """    
    df = pd.read_csv(table_path)
    selected = df[select_features]
    y_pred = trained_model.predict(selected)

    return pd.concat([df["ID"], pd.Series(y_pred, name='Rating')], axis=1)

#### Train and Predict - Bonus model
The following section uses the functions above to predict the raiting column in the given reviews tables

In [5]:
trained_model, selected_features = train_bonus_ass_model()
y_pred_rev_training =   bonus_ass_rating_prediction("Documents/reviews_training.csv", trained_model, selected_features)
y_pred_rev_rollout =    bonus_ass_rating_prediction("Documents/reviews_rollout.csv", trained_model, selected_features)


c:\Users\ofeks\OneDrive\תואר_שני\שנה ב\סמסטר א\נושאים מתקדמים במדעי הנתונים למנהל עסקים\temp_code\venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [1208 1527] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\ofeks\OneDrive\תואר_שני\שנה ב\סמסטר א\נושאים מתקדמים במדעי הנתונים למנהל עסקים\temp_code\venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


#### Merge tables
- ffp_rollout_X
- ffp_train

The merge process has one critical issue: Not all ID's in the ffp table, exist in the reviews table, That is when when performing an ```outer join```, the combined data will have ```Nan``` values in the ```Rating``` column.
We will deal with this issue in the next step.

In [6]:
df_ffp_train = pd.read_csv("Documents/ffp_train.csv")
merged_ffp_train = pd.merge(left=df_ffp_train, right=y_pred_rev_training,how="outer", on="ID")

df_ffp_rollout = pd.read_csv("Documents/ffp_rollout_X.csv")
merged_ffp_rollout = pd.merge(left=df_ffp_rollout, right=y_pred_rev_rollout,how="outer", on="ID")

## Step Two
### Choosing a model

In order to predict the ```Buyer flag``` of the ffp table, we will use the merged tables.
The problem: Some rows contain the rating value and some dont. To solve this, we will build model that is made out of two different models. One will work on the fpp table and one will work on the merged table. Predicted value will be chosen from one of the two results

### Model 1 - Table containing both rating and ffp data
Clear all ```Nan``` values from the table

In [7]:
# Devide the data according to the rating values.
# model1 will use df_ffp_data (data with no rating prediction)
# model2 will use df_combined_data (data with rating prediction)

#df_model1_train_no_rating = merged_ffp_train[merged_ffp_train["Rating"].isna()].drop("Rating", axis=1)
df_model2_train_both = merged_ffp_train[~merged_ffp_train["Rating"].isna()]


### Testing different models
We will try two algorithms and two feature selection methods:

In [8]:
# Strip ID and label
features = df_model2_train_both.drop(["BUYER_FLAG","ID"], axis=1)
# Extract the label column
label = df_model2_train_both["BUYER_FLAG"]


##### Feature selection methods
- One uses a correlation matrix to pick the most corralated features with the label.
- The second uses the function ```SelectPercentile```

In [9]:
def select_features_by_corr(data, treshold):
    mat = data.corr().drop("BUYER_FLAG", axis=0)
    mat = mat[abs(mat["BUYER_FLAG"]) > treshold]
    return list(mat.index)
    #REMOVE THE BUYER FLAG

from sklearn.feature_selection import SelectPercentile

def select_features_by_SelecPer(features, lable, p):
    selector = SelectPercentile(percentile=p)
    selector.fit_transform(features, label)
    return selector.get_feature_names_out()

Studying the label properties

In [10]:
data = df_model2_train_both.iloc[:,1:] # Get rid of ID col

mat = data.corr().drop("BUYER_FLAG", axis=0)
mat["BUYER_FLAG"] = abs(mat["BUYER_FLAG"])
print(f"Minimum corrlation value with 'BUYER_FLAG': {mat.describe()['BUYER_FLAG'].loc['min']}")
print(f"Maximum corrlation value with 'BUYER_FLAG': {mat.describe()['BUYER_FLAG'].loc['max']}")

Minimum corrlation value with 'BUYER_FLAG': 0.0031826178888089756
Maximum corrlation value with 'BUYER_FLAG': 0.560140516762669


Trying XGBoost with Selection by corr

In [11]:
from sklearn.metrics import f1_score

best_score = 0                         # Hold the F1 score of the best model
score = 0                              # Score of the current model
last_count = -1                        # flag for preventing reapetition
best_features = []
best_model_xgb_corr = ""
st = ""                                # Result string

for t in tqdm(range(30, 5600, 1)): # 0.03 Treshold includes all 22 features
 
    # Treshold is iterated from 0.003 to 0.56 increasing by 0.0001
    treshold = t / 10000
    selected_features = select_features_by_corr(data, treshold)

    # Break condition
    if len(selected_features) == last_count:
        continue
    else:
        last_count = len(selected_features)

    x_train, x_test, y_train, y_test = train_test_split(features[selected_features], label, test_size=0.2, random_state=1)
    for d in range(1, 10, 1):
        
        # Train XGBoost model
        classes_weights = class_weight.compute_sample_weight(
            class_weight='balanced',
            y=y_train
        )

        xgbclf = xgb.XGBClassifier(max_depth=d, seed=2)
        xgbclf.fit(x_train, y_train, sample_weight=classes_weights)
        y_pred = xgbclf.predict(x_test)
        score = f1_score(y_test, y_pred)
        # print(f"Current run:\nThresh: {treshold}\nVariables: {len(selected_features)}\n d: {d}\nF1: {score}")

        # Hold the best score
        if score > best_score:
            best_score = score
            best_features = selected_features
            best_model_xgb_corr = xgbclf
            st = f"The best XGBoost model...\nHas a treshold of {treshold}\nUses {len(selected_features)} features\nHas a model depth of {d}\nand an F1 score of {best_score}"
            st += f"\nFeatures selected: {best_features}"
print(40*"~", st, 40*"~", sep="\n", end="")

100%|██████████| 5570/5570 [00:21<00:00, 256.82it/s]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The best XGBoost model...
Has a treshold of 0.0261
Uses 10 features
Has a model depth of 2
and an F1 score of 0.6382978723404256
Features selected: ['STATUS_PANTINUM', 'STATUS_GOLD', 'STATUS_SILVER', 'FARE_L_Y1', 'FARE_L_Y4', 'FARE_L_Y5', 'POINTS_L_Y4', 'POINTS_L_Y5', 'COUPON_FLAG', 'Rating']
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Using Random Forest with selection by corr

In [12]:
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

best_score = 0                         # Hold the F1 score of the best model
score = 0                              # Score of the current model
last_count = -1                        # flag for preventing reapetition
best_features = []
best_model_rf_corr = ""
st = ""                                # Result string

for t in tqdm(range(30, 5600, 1)): # 0.03 Treshold includes all 22 features
 
    # Treshold is iterated from 0.003 to 0.56 increasing by 0.0001
    treshold = t / 10000
    selected_features = select_features_by_corr(data, treshold)

    # Break condition
    if len(selected_features) == last_count:
        continue
    else:
        last_count = len(selected_features)

    x_train, x_test, y_train, y_test = train_test_split(features[selected_features], label, test_size=0.2, random_state=1)
    for n_e in range(25, 200, 1):
        # Train XGBoost model
        rf = RandomForestClassifier(n_estimators=n_e, class_weight='balanced')
        rf.fit(x_train, y_train)
        y_pred = rf.predict(x_test)
        score = f1_score(y_test, y_pred)
        # print(f"Current run:\nThresh: {treshold}\nVariables: {len(selected_features)}\n d: {d}\nF1: {score}")

        # Hold the best score
        if score > best_score:
            best_score = score
            best_features = selected_features
            best_model_rf_corr = rf
            st = f"The best Random Forest model...\nHas a treshold of {treshold}\nUses {len(selected_features)} features\nHas {n_e} estimators\nand an F1 score of {best_score}"
            st += f"\nFeatures selected: {best_features}"


print(40*"~", st, 40*"~", sep="\n", end="")

  0%|          | 0/5570 [00:00<?, ?it/s]

We will now repeat the algorithmes above, using the select percentile function.

In [ ]:
from sklearn.metrics import f1_score

best_score = 0                         # Hold the F1 score of the best model
score = 0                              # Score of the current model
last_count = -1                        # flag for preventing reapetition
best_features_xgb_per = []
best_model_xgb_per = ""
st = ""                                # Result string

for p in tqdm(range(1, 101, 1)): 
 
    selected_features = select_features_by_SelecPer(features, label, p)

    # Break condition
    if len(selected_features) == last_count:
        continue
    else:
        last_count = len(selected_features)

    x_train, x_test, y_train, y_test = train_test_split(features[selected_features], label, test_size=0.2, random_state=1)
    for d in range(1, 10, 1):
        # Train XGBoost model
        classes_weights = class_weight.compute_sample_weight(
            class_weight='balanced',
            y=y_train
        )

        xgbclf = xgb.XGBClassifier(max_depth=d, seed=2)
        xgbclf.fit(x_train, y_train, sample_weight=classes_weights)
        y_pred = xgbclf.predict(x_test)
        score = f1_score(y_test, y_pred)
        # print(f"Current run:\nThresh: {treshold}\nVariables: {len(selected_features)}\n d: {d}\nF1: {score}")

        # Hold the best score
        if score > best_score:
            best_score = score
            best_features_xgb_per = selected_features
            best_model_xgb_per = xgbclf
            st = f"The best XGBoost model...\nHas a percentile of {p}\nUses {len(selected_features)} features\nHas a model depth of {d}\nand an F1 score of {best_score}"
            
print(40*"~", st, 40*"~", sep="\n", end="")

100%|██████████| 100/100 [00:09<00:00, 10.51it/s]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The best XGBoost model...
Has a percentile of 43
Uses 10 features
Has a model depth of 2
and an F1 score of 0.6382978723404256
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Now the RandomForest with the selectPercentile

In [ ]:
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

best_score = 0                         # Hold the F1 score of the best model
score = 0                              # Score of the current model
last_count = -1                        # flag for preventing reapetition
best_features_rf_per = []
best_model_rf_per = ""
st = ""                                # Result string

for p in tqdm(range(1, 101, 1)): 
 
    selected_features = select_features_by_SelecPer(features, label, p)

    # Break condition
    if len(selected_features) == last_count:
        continue
    else:
        last_count = len(selected_features)

    x_train, x_test, y_train, y_test = train_test_split(features[selected_features], label, test_size=0.2, random_state=1)
    for n_e in range(1, 200, 1):
        # Train XGBoost model
        rf = RandomForestClassifier(n_estimators=n_e, class_weight='balanced')
        rf.fit(x_train, y_train)
        y_pred = rf.predict(x_test)
        score = f1_score(y_test, y_pred)
        # print(f"Current run:\nThresh: {treshold}\nVariables: {len(selected_features)}\n d: {d}\nF1: {score}")

        # Hold the best score
        if score > best_score:
            best_score = score
            best_features_rf_per = selected_features
            best_model_rf_per = rf
            st = f"The best Random Forest model...\nHas a percentile of {p}\nUses {len(selected_features)} features\nHas {n_e} Estimators\nand an F1 score of {best_score}"
            
print(40*"~", st, 40*"~", sep="\n", end="")

100%|██████████| 100/100 [13:21<00:00,  8.02s/it]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The best Random Forest model...
Has a percentile of 15
Uses 4 features
Has 1 Estimators
and an F1 score of 0.6326530612244898
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

### Model 2 - Table containing ffp data only

Need to rerun models....

In [ ]:
df_ffp_train

,ID,CUSTOMER_GRADE,STATUS_PANTINUM,STATUS_GOLD,STATUS_SILVER,NUM_DEAL,LAST_DEAL,ADVANCE_PURCHASE,FARE_L_Y1,FARE_L_Y2,...,POINTS_L_Y1,POINTS_L_Y2,POINTS_L_Y3,POINTS_L_Y4,POINTS_L_Y5,COUPON_FLAG,CANCEL_FLAG,CREDIT_FLAG,RELATED_FLAG,BUYER_FLAG
0,1,11.545711,0,0,0,3,29.3,24,82.7,92.7,...,71.3,60.7,98.1,81.6,78.6,0,0,0,0,0
1,2,9.493459,0,0,0,5,26.7,27,120.7,112.0,...,126.3,112.5,104.6,110.9,137.3,0,0,1,0,0
2,3,3.882457,0,0,0,4,26.7,9,97.3,92.0,...,107.7,120.7,62.5,63.9,92.9,0,0,0,0,0
3,4,9.511699,0,0,0,2,61.3,24,123.3,130.7,...,129.5,132.1,135.1,104.1,118.2,0,0,0,0,0
4,5,14.798792,0,0,0,4,37.3,20,136.0,135.3,...,145.7,124.5,122.7,140.1,130.9,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,8.993560,0,0,0,6,16.0,25,98.7,92.7,...,57.2,89.1,83.5,56.1,79.8,0,0,0,0,0
29996,29997,9.809759,0,1,0,2,117.3,20,228.0,233.3,...,178.9,215.5,214.1,216.7,232.6,0,0,0,0,0
29997,29998,11.226843,0,0,0,4,24.0,21,84.7,87.3,...,53.0,83.4,81.3,73.9,90.6,0,0,0,0,0
29998,29999,12.926248,0,0,0,3,42.7,20,121.3,126.7,...,98.9,80.0,89.0,121.5,158.5,0,0,0,0,0


In [ ]:
features = df_ffp_train.drop(["ID", "BUYER_FLAG"], axis=1)
label = df_ffp_train["BUYER_FLAG"]

x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=1)

NameError: name 'df_model1_train_no_rating' is not defined

In [ ]:
def get_best_xgboost(features, label):
    
    parameters = {'score': 0,
                  'features': [],
                  'model': [],
                  'depth': -1
                  }        
    
    x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=1)

    for d in range(1, 10, 1):
        classes_weights = class_weight.compute_sample_weight(
            class_weight='balanced',
            y=y_train
        )

        xgbclf = xgb.XGBClassifier(max_depth=d, seed=2)
        xgbclf.fit(x_train, y_train, sample_weight=classes_weights)
        y_pred = xgbclf.predict(x_test)
        score = f1_score(y_test, y_pred)
                
        if score > parameters['score']:
            parameters['score'] = score
            parameters['features'] = selected_features
            parameters['model'] = xgbclf
            parameters['depth'] = d
            #st = f"The best XGBoost model...\nHas a treshold of {treshold}\nUses {len(selected_features)} features\nHas a model depth of {d}\nand an F1 score of {best_score}"
            #st += f"\nFeatures selected: {best_features}"
            
    return parameters

In [ ]:
def get_best_rf(features, label):

    parameters = {'score': 0,
                  'features': [],
                  'model': [],
                  'n_estimators': -1
                } 
    
    x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=1)
    
    for n_e in range(1, 200, 1):

        rf = RandomForestClassifier(n_estimators=n_e, class_weight='balanced')
        rf.fit(x_train, y_train)
        y_pred = rf.predict(x_test)
        score = f1_score(y_test, y_pred)

        if score > parameters['score']:
            parameters['score'] = score
            parameters['features'] = selected_features
            parameters['model'] = rf
            parameters['n_estimators'] = n_e
            
    return parameters    

In [ ]:
prev = -1
params = {}
best_params = {}

for i in tqdm(range(1, 500, 1)):
    treshold = i / 1000
    selected = select_features_by_corr(df_ffp_train,treshold)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_xgboost(features[selected], label)
    if params['score'] > best_params['score']:
        best_params = params

params_xgb_corr = best_params
params_xgb_corr

In [ ]:
prev = -1
params = {}
best_params = {}

for p in tqdm(range(1, 101, 1)):
    selected = select_features_by_SelecPer(features, label, p)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_xgboost(selected, label)
    if params['score'] > best_params['score']:
        best_params = params

params_xgb_per = best_params
params_xgb_per

In [ ]:
prev = -1
params = {}
best_params = {}

for i in tqdm(range(1, 500, 1)):
    treshold = i / 1000
    selected = select_features_by_corr(df_ffp_train,treshold)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_rf(features[selected], label)
    if params['score'] > best_params['score']:
        best_params = params

params_rf_corr = best_params
params_rf_corr

In [ ]:
prev = -1
params = {}
best_params = {}

for p in tqdm(range(1, 101, 1)):
    selected = select_features_by_SelecPer(features, label, p)

    if prev == len(selected):
        continue
    prev = len(selected)

    params = get_best_rf(selected, label)
    if params['score'] > best_params['score']:
        best_params = params

params_rf_per = best_params
params_rf_per

### Choosing the "Winning" models


In [ ]:
model1 = ''
model2 = ''

recommandations = pd.DataFrame(columns="BUYER_FLAG")

def combined_model(row):
    x = row.drop(["BUYER_FLAG"], axis=1)
    if x["Rating"].isna():
        return model2.predict(x.drop(["Rating"], axis=1))
    return model1.predict(x)

recommandations["BUYER_FLAG"] = pd.apply(data=merged_ffp_rollout, combined_model)